<a href="https://colab.research.google.com/github/san-jay29/san-jay29/blob/main/cat%20and%20dog%20cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models


In [7]:
import zipfile
import os

with zipfile.ZipFile('/content/cat ds.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

print(os.listdir('/content/cat ds'))

Zip file found at: /content/cat ds.zip
Dataset extracted to: /content
Contents of /content after extraction:
['.config', 'val.csv', 'cat ds.zip', 'val', 'train.csv', 'train', 'sample_data']


In [11]:
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=20
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/val',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 275 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


In [15]:
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False


In [17]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])


In [18]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [19]:
initial_epochs = 5

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=initial_epochs
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7200 - loss: 0.5562 - val_accuracy: 0.9571 - val_loss: 0.0829
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9733 - loss: 0.1388 - val_accuracy: 0.9714 - val_loss: 0.0399
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.9663 - loss: 0.0702 - val_accuracy: 1.0000 - val_loss: 0.0218
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9743 - loss: 0.0636 - val_accuracy: 0.9857 - val_loss: 0.0436
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9877 - loss: 0.0399 - val_accuracy: 0.9857 - val_loss: 0.0212


In [20]:
base_model.trainable = True

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [22]:
fine_tune_epochs = 5
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1]
)


Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 0.8825 - loss: 0.3822 - val_accuracy: 1.0000 - val_loss: 0.0169
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.8284 - loss: 0.6065 - val_accuracy: 1.0000 - val_loss: 0.0133
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.8445 - loss: 0.3714 - val_accuracy: 1.0000 - val_loss: 0.0101
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.9534 - loss: 0.1185 - val_accuracy: 1.0000 - val_loss: 0.0082
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.9097 - loss: 0.2614 - val_accuracy: 1.0000 - val_loss: 0.0074
Epoch 10/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.9324 - loss: 0.1637 - val_accuracy: 1.0000 - val_loss: 0.0067


In [26]:
model.save('my_model.keras')

In [29]:
loss, acc = model.evaluate(val_generator)
print(f"\n Final Validation Accuracy: {acc * 100:.2f}%")


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 804ms/step - accuracy: 1.0000 - loss: 0.0073

 Final Validation Accuracy: 100.00%
